In [1]:
# Google Colab specific
# %cd ~/../content
# !rm -rf openet

# !git clone https://github.com/aetriusgx/openet.git
# %cd openet

In [5]:
from datetime import datetime
from pathlib import Path
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns

sns.set_theme()

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
historical = pd.read_csv('./data/monterey_historical.csv', low_memory=False)
historical['time'] = pd.to_datetime(historical['time'])
historical.info()

In [ ]:
# Gather current forecast data for the county
forecasting_table = pd.DataFrame()
files = Path(f"data/forecasts/monterey/").glob("*.csv")

for file in files:
    # splits into [$date, 'forecast.csv']
    parts = str(file.name).split("_")
    data = pd.read_csv(file, low_memory=False)
    data["forecasting_date"] = parts[0]
    forecasting_table = pd.concat([data, forecasting_table], ignore_index=True)

forecasting_table['forecasting_date'] = pd.to_datetime(forecasting_table['forecasting_date'])
forecasting_table['time'] = pd.to_datetime(forecasting_table['time'])
forecasting_table.info()

In [ ]:
dt = historical.loc[(historical['time'].dt.year == 2024), :]
dt = dt.merge(forecasting_table, on=['field_id', 'time', 'crop'], how='right').set_index(['forecasting_date', 'field_id', 'crop', 'time']).reset_index()
dt

In [ ]:
# Add additional data to the data table
monterey_points = pd.read_csv("./Monterey.csv", low_memory=False).set_index("OPENET_ID").rename_axis("field_id")

# Expand .geo column into lon, lat columns
monterey_geo = (monterey_points[".geo"]
                .apply(lambda x: pd.Series(dict(json.loads(x))))['coordinates']
                .apply(lambda x: pd.Series(list(x), index=['longitude', 'latitude'])))
monterey_geo.info()

In [ ]:
dt = dt.join(monterey_geo, how="left", on=["field_id"], validate="many_to_one")
dt.info()

In [ ]:
# Add crop data
cdl_codes = pd.read_csv("cdl_codes.csv", low_memory=False).set_index("Codes")

dt = dt.join(cdl_codes, how="left", on="crop", validate="many_to_many")
dt.info()

In [ ]:
forecast_dates = forecasting_table['forecasting_date'].unique()
fields = dt['field_id'].unique()
crops = dt['crop'].unique()

# Analysis

## Helpers
Below are functions that are being used to calculate data and generate plots.

### calculate_metrics
This function calculate the mean absolute error (mae), root mean squared error (rmse), mean forecast error (bias), correlation coefficient (R), and skill score.

MAE, RMSE, and R are calculated using sklearn's metric module.

Skill score is calculated by getting the climatology for each field within the input's date range.
* Negative skill scores indicate the MSE for forecast is larger than the MSE for climatology
* Positive skill scores indicate otherwise

The function is very flexible given the data is formatted appropriately. It has the option of enabling normalization which is based on the average specified variable (ET, ETo, or ETof) throughout that field's historical data.

In [ ]:
def calculate_metrics(data: pd.DataFrame, *, historical: pd.DataFrame, actual: str, expected: str, normalize: bool = False) -> pd.Series:
	# Calculate error metrics
	mae = mean_absolute_error(data[actual], data[expected])
	forecast_mse = mean_squared_error(data[actual], data[expected])

	rmse = np.sqrt(forecast_mse)
	# R2 calculates how predictable the outcome is with the input.
	# Positive scores indicate the input correlates to a proportional output.
	# Negative scores indicate a worsening correlation.
    # r2_score(data[actual], data[expected], force_finite=True)
	cor = np.corrcoef(data[actual], data[expected])[0, 1]
	bias = np.mean(data[actual] - data[expected])

	# Climatology uses the mean of actual_et for that time of year using historical data.
	# Trim down historical reference to just the field, and crops provided
	field = data.head(1).squeeze()
	field_crop_table = historical[(historical['field_id'] == field['field_id']) & (historical['crop'] == field['crop'])]

	# Create a column for day of the year
	field_crop_table['doy'] = field_crop_table['time'].dt.dayofyear

	# Trim down historical reference table so only dates provided by the input DataFrame are calculated
	start_date = data['time'].min()
	end_date = data['time'].max()
	historical_period = field_crop_table[(field_crop_table['doy'] >= start_date.dayofyear) & (field_crop_table['doy'] <= end_date.dayofyear)]

	# Produce table of historical averages for current index
	climatology = historical_period.groupby([historical_period['doy']])[actual].agg('mean')
	climatology_mse = mean_squared_error(data[actual], climatology)

	# Negative skill scores indicate the MSE for forecast is larger than the MSE for climatology
	# Positive skill scores indicate otherwise
	skill_score = 1 - (forecast_mse / climatology_mse)

	if normalize:
		mean_variable = historical[historical['field_id'] == field['field_id']][actual].mean()
		mae = mae / mean_variable
		rmse = np.sqrt(forecast_mse / mean_variable)
		bias = bias / mean_variable

	return pd.Series({
		'mae': mae.round(2),
		'rmse': rmse.round(2),
		'corr': cor.round(2),
		'bias': bias.round(2),
		'skill_score': skill_score.round(2)
	})

### eval_metrics
This function evaluates the metrics for each variable. The output is a DataFrame containing the metrics with a column specifying which variable (ET, ETo, ETof)

In [ ]:
def eval_metrics(table: pd.DataFrame, **kwargs) -> pd.DataFrame:
    metrics_table = pd.DataFrame(columns=["field_id", "crop", "variable", "mae", "rmse", "corr", "bias", "skill_score"])

    et_metrics = table.reset_index().groupby(['field_id', 'crop']).apply(calculate_metrics, historical=historical,
                                                                      actual='actual_et', expected='expected_et', **kwargs).reset_index()
    et_metrics['variable'] = "ET"
    metrics_table = pd.concat([et_metrics.astype(metrics_table.dtypes), metrics_table.astype(et_metrics.dtypes)], ignore_index=True)

    eto_metrics = table.reset_index().groupby(['field_id', 'crop']).apply(calculate_metrics, historical=historical,
                                                                      actual='actual_eto', expected='expected_eto', **kwargs).reset_index()
    eto_metrics['variable'] = "ETo"
    metrics_table = pd.concat([eto_metrics.astype(metrics_table.dtypes), metrics_table.astype(eto_metrics.dtypes)], ignore_index=True)

    etof_metrics = table.reset_index().groupby(['field_id', 'crop']).apply(calculate_metrics, historical=historical,
                                                                      actual='actual_etof', expected='expected_etof', **kwargs).reset_index()
    etof_metrics['variable'] = "ETof"
    metrics_table = pd.concat([etof_metrics.astype(metrics_table.dtypes), metrics_table.astype(etof_metrics.dtypes)], ignore_index=True)

    return metrics_table

### timeseries_plot
This plot function is designed to easily showcase actual vs forecast data. It is also flexible in that it can show a singular line if a second column is not specified. This is used to show the metrics over time.

In [ ]:
def timeseries_plot(data, *, x, y_a, y_b=None, title='', ylabel, xlabel, figsize=(6, 4), as_percent=False, **kwargs):
    figure, xa = plt.subplots(figsize=figsize)
    xb = xa.twinx() if y_b else None

    if y_a:
        first_line = sns.lineplot(data, x=x, y=y_a, ax=xa, errorbar=None, **kwargs)
    if y_b:
        second_line = sns.lineplot(data, x=x, y=y_b, ax=xb, linestyle='dashed', errorbar=None, legend=False, **kwargs)

    # Remove y axis label for the second line
    if xb: xb.set_ylabel('')
    # first_line is the main subplot so metadata should get changed for that one
    # Relabel y axis
    if ylabel: xa.set_ylabel(ylabel)
    if as_percent: xa.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    # Relabel x axis
    if xlabel: xa.set_xlabel(xlabel)
    # Rotate x-time axis
    first_line.tick_params(axis='x', rotation=90)
    # Relocate legend to be outside subplot
    sns.move_legend(first_line, "upper left", bbox_to_anchor=(1.25, 1))
    # Set title if specified
    first_line.set_title(title)

    figure.show()
    return (figure, xa, xb)

## Calculation

Est. run time: 41 min.

In [ ]:
metrics = dt[(dt['time'] > dt['forecasting_date']) & (~dt['actual_et'].isna())].groupby(['forecasting_date']).apply(eval_metrics)
# metrics = pd.read_csv('data/metrics/monterey_metrics.csv', low_memory=False)
# metrics['forecasting_date'] = pd.to_datetime(metrics['forecasting_date'])
# metrics = metrics.drop(columns=['Unnamed: 1'])
# metrics = metrics.set_index(['forecasting_date'])
metrics.head()

In [ ]:
metrics.to_csv('data/metrics/monterey_metrics.csv', index=False)

Est. run time: 2h 30m

In [ ]:
metrics_norm = dt[(dt['time'] > dt['forecasting_date']) & (~dt['actual_et'].isna())].groupby(['forecasting_date']).apply(eval_metrics, normalize=True)
# metrics_norm = pd.read_csv('data/metrics/monterey_metrics_normalized.csv', low_memory=False)
# metrics_norm['forecasting_date'] = pd.to_datetime(metrics_norm['forecasting_date'])
# metrics_norm = metrics_norm.drop(columns=['Unnamed: 1'])
# metrics_norm = metrics_norm.set_index(['forecasting_date'])
metrics_norm

In [ ]:
metrics_norm.to_csv('data/metrics/monterey_metrics_normalized.csv', index=False)

### Crop metrics

In [ ]:
crop_metrics = dt[(dt['time'] > dt['forecasting_date']) & (~dt['actual_et'].isna())].groupby(['forecasting_date', 'crop']).apply(eval_metrics, normalize=True)

## Best in categories

In [ ]:
grouped_stats = metrics_norm.groupby(['field_id', 'variable'])[['mae', 'rmse', 'bias', 'skill_score']].agg('mean').round(2)
grouped_stats.head()

In [ ]:
best_ss_field = grouped_stats[grouped_stats['skill_score'] == grouped_stats['skill_score'].max()]
best_ss_field = list(best_ss_field.index.get_level_values(0))
best_ss_field

# Plotting The Metrics

In [ ]:
field_limitn = 1
date_limit = datetime(year=2024, month=7, day=19)

plotter = metrics_norm[(metrics_norm['field_id'].isin(fields.sample(10))) &
                       (metrics_norm.index.get_level_values(0) < date_limit)]

In [ ]:
timeseries_plot(plotter, x='forecasting_date', y_a='corr',
                hue='field_id', title='Correlation Coeff (R^2) Progression Over Time', style='variable',
                xlabel=' ', ylabel='R', figsize=(12, 4));

In [ ]:
timeseries_plot(plotter, x='forecasting_date', y_a='mae',
                hue='field_id', title='MAE Progression Over Time', style='variable',
                xlabel=' ', ylabel='% Error', as_percent=True, figsize=(12, 4));

In [ ]:
timeseries_plot(plotter, x='forecasting_date', y_a='rmse',
                hue='field_id', title='RMSE Progression Over Time', style='variable',
                xlabel=' ', ylabel='% Error', as_percent=True, figsize=(12, 4));

In [ ]:
timeseries_plot(plotter, x='forecasting_date', y_a='bias',
                hue='field_id', title='Bias Over Time', style='variable',
                xlabel=' ', ylabel='Bias', as_percent=True, figsize=(12, 4));

In [ ]:
timeseries_plot(plotter, x='forecasting_date', y_a='skill_score',
                hue='field_id', title='Forecasting Performance Over Time', style='variable',
                xlabel=' ', ylabel='Skill Score', figsize=(10, 4));

In [ ]:
plt.figure(figsize=(10, 6))
metrics = ['mae', 'rmse', 'bias', 'skill_score']
corr_matrix = metrics_norm[metrics].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap of Metrics')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x='mae', y='rmse', data=metrics_norm, hue='crop', palette='tab10')
plt.title('Scatter Plot of MAE vs. RMSE')
plt.xlabel('Mean Absolute Error (MAE)')
plt.ylabel('Root Mean Square Error (RMSE)')
plt.legend(title='Crop')
plt.show()

In [ ]:
# Box plot for distribution of metrics
metrics = ['mae', 'rmse', 'bias', 'skill_score']
plt.figure(figsize=(12, 6))
sns.boxplot(plotter[metrics])
plt.title('Distribution of Forecasting Metrics')
plt.xlabel('Metrics')
plt.ylabel('Values')
plt.show()

## Spatial Plotting

First merge the geo table created in the beginning with the metrics table

In [ ]:
metrics_norm = metrics_norm.join(monterey_geo, how='left', on='field_id', validate='many_to_one')

In [ ]:
metrics_norm.info()

In [ ]:
metrics_norm.plot(x="longitude", y="latitude", kind="scatter", c="skill_score",
        colormap="YlOrRd");